In [1]:
%use dataframe
%use multik

In [2]:
import kotlin.math.ln
import kotlin.math.min
import kotlin.math.exp
import kotlin.random.Random

In [237]:
val df = dataFrameOf(
    "X1" to List(100) { 1 },
    "X2" to List(100) { if (it<50) 1 else 0 }
).add("y") {
    val p= 1/(1+exp(0.5*"X1"<Int>().toFloat() - 0.5*"X2"<Int>().toFloat()))
    if (Random.nextFloat()<p) 1 else 0
}
df.head(10)

X1,X2,y
1,1,0
1,1,1
1,1,0
1,1,0
1,1,0
1,1,0
1,1,0
1,1,0
1,1,1
1,1,1


In [116]:
df.writeCSV("testdata.csv")

In [6]:
val totals = df2.groupBy { group }.aggregate { count() into "total" }

In [7]:
totals

group,total
0,50
1,50


In [19]:
totals.filter { group == 1 }.total.toList()

[50]

In [4]:
val theta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> Double =  { beta,X ->  X.dot(beta)[0,0] }
val dtheta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> NDArray<Double, D2> =  { beta,X ->  X }
val ddtheta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> NDArray<Double, D2> =  { beta,X ->  mk.zeros<Double>(beta.shape[0], beta.shape[0]) }

val b: (theta: Double) -> Double =  { theta ->  ln(1+exp(theta)) }
val db: (theta: Double) -> Double =  { theta ->  exp(theta)/(1+exp(theta)) }
val ddb: (theta: Double) -> Double =  { theta ->  exp(theta)/((1+exp(theta)).pow(2.0)) }

In [62]:
val theta: (beta: DoubleArray, X: DoubleArray) -> Double =  { beta,X ->  beta.foldIndexed(initial=0.0){i,acc,el -> acc + el*X[i] } }
val dtheta: (beta: DoubleArray, X: DoubleArray) -> DoubleArray =  { beta,X ->  X }
val ddtheta: (beta: DoubleArray, X: DoubleArray) -> DoubleArray =  { beta,X ->  DoubleArray(beta.size * beta.size) {0.0} }

val b: (theta: Double) -> Double =  { theta ->  ln(1+exp(theta)) }
val db: (theta: Double) -> Double =  { theta ->  exp(theta)/(1+exp(theta)) }
val ddb: (theta: Double) -> Double =  { theta ->  exp(theta)/((1+exp(theta)).pow(2.0)) }

In [59]:
val theta: (beta: NDArray<Double, D2>, X: NDArray<Double, D2>) -> Double =  { beta,X ->  X.dot(beta)[0,0] }

In [61]:
mk.ones<Double>(3, 4)

[[1.0, 1.0, 1.0, 1.0],
[1.0, 1.0, 1.0, 1.0],
[1.0, 1.0, 1.0, 1.0]]

In [46]:
df.sumOf { (y.toDouble() -db(theta(0.0))) *dtheta(0.0) }

23.0

In [242]:
val y=mk.ndarray(df.y.toList().map { it.toDouble()} ,df.count(),1)
val df2=df.add("X") {
    doubleArrayOf(X1.toDouble(),X2.toDouble())
}
    .remove {X1 and X2}
    .convert{ y }.with { it.toDouble() }

In [248]:
var beta=doubleArrayOf(1.0,0.0)
while(true) {
    val l = df2.sumOf { -b(theta(beta, it.X)) + y*theta(beta, it.X) }
    
    val dl_ = mk.ndarray(df2.sumOfArray { 
        val dtheta_ = dtheta(beta, it.X)
        val db_ = db(theta(beta, it.X))
        DoubleArray(dtheta_.size) {(y-db_) * dtheta_[it]}
    },1,beta.size)
    
    val hessian = try {
        mk.linalg.inv(
        mk.ndarray(df2.sumOfArray {
            val db_ = db(theta(beta, it.X))
            val ddtheta_ = ddtheta(beta, it.X)
            val ddb_of_theta_id = ddb(theta(beta, it.X))
            val dtheta_id = dtheta(beta, it.X)
            DoubleArray(beta.size*beta.size) {
                (y-db_) * ddtheta_[it] - 
                ddb_of_theta_id* dtheta_id[it % beta.size] * dtheta_id[it / beta.size]
                }
            },beta.size,beta.size)
        )
    } catch(e: Exception) {
        println("Hessian is not invertible  - ${e.message}")
        break
    }
    val dif = dl_.dot(hessian).toDoubleArray()
    val dif_norm = mk.linalg.norm(mk.ndarray(dif,beta.size,1))
    beta = beta.elementMinus(dif)
    if (dif_norm<1e-4) break
    println("likelihood: $l, change = $dif_norm") 
}
println(beta.joinToString())

likelihood: -91.32616875182218, change = 2.0805247891848104
likelihood: -67.41540206865687, change = 0.3708966396963517
likelihood: -66.55199664836803, change = 0.02981560135156381
likelihood: -66.54896185469012, change = 1.1557575992833986E-4
-0.6632942174102644, 0.502951567335085


In [51]:
inline fun <reified T> toArray(list: List<*>): Array<T> {
    return (list as List<T>).toTypedArray()
}

In [258]:
//Canonical link:
var beta=doubleArrayOf(1.0,0.0)
while(true) {
    val l = df2.sumOf { -b(theta(beta, it.X)) + y*theta(beta, it.X) }
    
    val dl_ = mk.ndarray(df2.sumOfArray { 
        val db_ = db(theta(beta, it.X))
        DoubleArray(beta.size) {(y-db_) * X[it]}
    },1,beta.size)
    
    val hessian = try {
        mk.linalg.inv(
        mk.ndarray(df2.sumOfArray {
            val ddb_of_theta_id = ddb(theta(beta, it.X))
            DoubleArray(beta.size*beta.size) {
               - ddb_of_theta_id* X[it % beta.size] * X[it / beta.size]
                }
            },beta.size,beta.size)
        )
    } catch(e: Exception) {
        println("Hessian is not invertible  - ${e.message}")
        break
    }
    val dif = dl_.dot(hessian).toDoubleArray()
    val dif_norm = mk.linalg.norm(mk.ndarray(dif,beta.size,1))
    beta = beta.elementMinus(dif)
    if (dif_norm<1e-4) break
    println("likelihood: $l, change = $dif_norm") 
}
println(beta.joinToString())

likelihood: -91.32616875182218, change = 2.0805247891848104
likelihood: -67.41540206865687, change = 0.3708966396963517
likelihood: -66.55199664836803, change = 0.02981560135156381
likelihood: -66.54896185469012, change = 1.1557575992833986E-4
-0.6632942174102644, 0.502951567335085


In [30]:
fun DoubleArray.elementPlus(other: DoubleArray): DoubleArray {
    return ( DoubleArray(this.size) { it -> this[it] + other [it]})
}
fun DoubleArray.elementMinus(other: DoubleArray): DoubleArray {
    return ( DoubleArray(this.size) { it -> this[it] - other [it]})
}

In [26]:
inline fun <T> Iterable<T>.sumOfArray(selector: (T) -> DoubleArray): DoubleArray {
    if (this.none()) return DoubleArray(1) { 0.0 }
    //return DoubleArray(1) {0.0}
    var sum = DoubleArray(selector(this.first()).size) { 0.0 }
    for (element in this) {
        //sum = sum.elementPlus(selector(element))    
        for (i in sum.indices) {
            sum[i] += selector(element)[i]
        }
    }
    return sum
}

In [167]:
fun <T> DataFrame<T>.sumOfArray(expression: RowExpression<T,DoubleArray>): DoubleArray =
    rows().sumOfArray { expression(it, it) }

In [244]:
List(4) { it}.sumOfArray { it1 -> DoubleArray(3) { it2 -> it2.toDouble()*it1 }  }


[0.0, 6.0, 12.0]

In [168]:
df.sumOfArray { DoubleArray(2){ it.toDouble()} }

[0.0, 100.0]

In [150]:
df.rows().sumOfArray { DoubleArray(2){ it.toDouble()} }

[0.0, 100.0]

In [15]:
DoubleArray(3) { it2 -> it2.toDouble() } + DoubleArray(3) { it2 -> it2.toDouble() } 

[0.0, 2.0, 4.0]